# RFSoC Offload Overlay 10G TX Example
Load the QSFP offload overlay

In [ ]:
import time
import sys
from rfsoc_qsfp_offload.overlay import Overlay

board_ip = '192.168.4.99'
client_ip = '192.168.4.1'

print("Initializing RFSoC QSFP Offload Overlay")
ol = Overlay(ignore_version=True)

# Wait for overlay to initialize
time.sleep(5) 

Configure the DAC

In [ ]:
DAC_TILE = 0       # DAC Tile 228
DAC_BLOCK = 0       # DAC Block 0
DAC_SAMPLE_FREQUENCY = 1024  # MSps
DAC_PLL_FREQUENCY = 491.52   # MHz
DAC_INTERP = 16 

ol.initialise_dac(tile=DAC_TILE,
                  block=DAC_BLOCK,
                  pll_freq=DAC_PLL_FREQUENCY,
                  fs=DAC_SAMPLE_FREQUENCY
                 )

ol.rfdc.dac_tiles[DAC_TILE].blocks[DAC_BLOCK].InterpolationFactor = DAC_INTERP

The RF-DAC is set up with an output sample rate of 1024 Msps and 16x interpolation. This means that the signal we generate requires a sample rate of $f_{s DAC}/16$. 

Next we can generate a 64 MHz sine wave using the `signal_generator` module supplied with this design.

In [ ]:
from rfsoc_offload import signal_generator

sin_data = signal_generator.sine(f=64e6, fs=DAC_SAMPLE_FREQUENCY*1e6/4)
sin_data = signal_generator.convert_to_int16(sin_data)
import plotly.graph_objs as go
go.FigureWidget(go.Scatter(y=sin_data[0:256]))

Create a Numpy array representing the interleaved I and Q components of the transmit signal and transfer to DMA

In [ ]:
import numpy as np
from pynq import allocate

zero_data = np.zeros(sin_data.size, np.int16)
interleaved = np.empty((sin_data.size + zero_data.size,), dtype=sin_data.dtype)
interleaved[0::2] = sin_data
interleaved[1::2] = zero_data

tx_buffer = allocate(shape=(interleaved.size,), dtype=np.int16)
tx_buffer[:] = interleaved

# Save buffer to file
tx_file = open("tx_signal.bin", mode='wb')
tx_buffer.tofile(tx_file)

ol.axi_dma_dac.sendchannel.transfer(tx_buffer, cyclic=True)

### Set the frequency of the output Mixer for the DAC channel

In [ ]:

ol.rfdc.dac_tiles[0].blocks[0].MixerSettings['Freq'] = 0 # MHz

### Stop the DMA transfer

In [ ]:
ol.axi_dma_dac.sendchannel.stop()